In [ ]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.optimizers import RMSprop

In [ ]:
import zipfile
from google.colab import drive

drive.mount('/content/drive/')



Mounted at /content/drive/


In [ ]:
train_dir ='/content/drive/MyDrive/capstone/new data/train'
valid_dir = '/content/drive/MyDrive/capstone/new data/valid'

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1/255)
train_generator= train_datagen.flow_from_directory(train_dir,target_size = (150,150),class_mode ='binary')
valid_datagen = ImageDataGenerator(rescale = 1/255)
valid_generator= valid_datagen.flow_from_directory(valid_dir,target_size = (150,150),class_mode ='binary')




Found 6499 images belonging to 2 classes.
Found 1849 images belonging to 2 classes.


In [ ]:
inceptionv3 = 'https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'
urllib.request.urlretrieve(inceptionv3, 'inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5')
local_weights_file = 'inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'

pre_trained_model =  InceptionV3(input_shape = (150,150,3), include_top = False, weights = None)
pre_trained_model.load_weights(local_weights_file)

for layer in pre_trained_model.layers:
  layer.trainable = False
last_layer =  pre_trained_model.get_layer('mixed7')

In [ ]:
x = layers.Flatten()(last_layer.output)
x = layers.Dense(512, activation ='relu')(x)
x = layers.Dense(1, activation='sigmoid')(x)


In [ ]:
model = Model(pre_trained_model.input, x)

In [ ]:
model.compile(optimizer=RMSprop(lr=0.0001),
                  loss='binary_crossentropy',
                  metrics=['acc'])

In [ ]:
model.fit(train_generator, validation_data = valid_generator, steps_per_epoch = 5, epochs = 10, verbose = 1)

Epoch 1/10
5/5 [==============================] - 286s 67s/step - loss: 6.3329 - acc: 0.4938 - val_loss: 0.5350 - val_acc: 0.7242
Epoch 2/10
5/5 [==============================] - 95s 23s/step - loss: 0.8003 - acc: 0.7125 - val_loss: 0.4544 - val_acc: 0.7783
Epoch 3/10
5/5 [==============================] - 89s 22s/step - loss: 0.4506 - acc: 0.8000 - val_loss: 0.3643 - val_acc: 0.8421
Epoch 4/10
5/5 [==============================] - 89s 22s/step - loss: 0.4965 - acc: 0.7750 - val_loss: 0.3557 - val_acc: 0.8442
Epoch 5/10
5/5 [==============================] - 83s 20s/step - loss: 0.3466 - acc: 0.8687 - val_loss: 0.3948 - val_acc: 0.8064
Epoch 6/10
5/5 [==============================] - 82s 20s/step - loss: 0.3664 - acc: 0.8188 - val_loss: 0.4442 - val_acc: 0.7994
Epoch 7/10
5/5 [==============================] - 89s 22s/step - loss: 0.3181 - acc: 0.8813 - val_loss: 0.5208 - val_acc: 0.7491
Epoch 8/10
5/5 [==============================] - 91s 22s/step - loss: 0.6429 - acc: 0.7125 - va

In [ ]:
#model lain

model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(16, (3,3), activation = 'relu', padding ='same', input_shape = (150,150,3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32, (3,3), activation = 'relu', padding ='same'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation = 'relu', padding ='same'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation ='relu'),
    tf.keras.layers.Dense(64, activation = 'relu'),
    tf.keras.layers.Dense(1, activation = 'sigmoid')
])

In [ ]:
model.compile(loss = 'binary_crossentropy', optimizer = RMSprop(learning_rate =0.001), metrics = ['accuracy'])
model.fit(train_generator, validation_data = valid_generator, epochs=10, verbose=1)

Epoch 1/20
204/204 [==============================] - 3998s 19s/step - loss: 0.6672 - accuracy: 0.6609 - val_loss: 0.6068 - val_accuracy: 0.6674
Epoch 2/20
204/204 [==============================] - 250s 1s/step - loss: 0.4326 - accuracy: 0.8101 - val_loss: 0.3918 - val_accuracy: 0.8237
Epoch 3/20
204/204 [==============================] - 244s 1s/step - loss: 0.3384 - accuracy: 0.8631 - val_loss: 0.3297 - val_accuracy: 0.8659
Epoch 4/20
204/204 [==============================] - 259s 1s/step - loss: 0.2702 - accuracy: 0.8908 - val_loss: 0.3664 - val_accuracy: 0.8588
Epoch 5/20
204/204 [==============================] - 268s 1s/step - loss: 0.2101 - accuracy: 0.9148 - val_loss: 0.3428 - val_accuracy: 0.8643
Epoch 6/20
204/204 [==============================] - 241s 1s/step - loss: 0.1593 - accuracy: 0.9363 - val_loss: 0.3293 - val_accuracy: 0.8837
Epoch 7/20
204/204 [==============================] - 255s 1s/step - loss: 0.1196 - accuracy: 0.9566 - val_loss: 0.3176 - val_accuracy: 0.88

In [ ]:
from google.colab import files
from tensorflow.keras.utils import load_img, img_to_array
import numpy as np

uploaded = files.upload()

for fn in uploaded.keys():

  # predicting images
  path = '/content/' + fn
  img = load_img(path, target_size=(150, 150))
  x = img_to_array(img)
  x /= 255
  x = np.expand_dims(x, axis=0)

  images = np.vstack([x])
  classes = model.predict(images, batch_size=10)
  print(classes[0])
  if classes[0]>0.5:
    print(fn + " is a fresh")
  else:
    print(fn + " is a rotten")
